In [3]:
import numpy as np
import math
import seaborn as sns

In [4]:
init_state = np.array([
    [0, 0, 0],
    [0, 0, 0],
    [0, 0, 0]
])

In [5]:
class Game:
    def __init__(self, state):
        self.state = state
        self.empty = self.make_empty(state)
        
    def make_empty(self, state):
        emp = []
        for i in range(3):
            for j in range(3):
                if state[i][j] == 0:
                    emp.append(3*i + j)
        
        return emp
    
    def is_lose(self, a):
        opp = 2 - (a-1)
        
        for i in range(3):
            if self.state[i][0] == self.state[i][1] == self.state[i][2] == opp:
                return 1
            elif self.state[0][i] == self.state[1][i] == self.state[2][i] == opp:
                return 1
        if self.state[0][0] == self.state[1][1] == self.state[2][2] == opp:
            return 1
        return 0
    
    def is_win(self, a):       
        for i in range(3):
            if self.state[i][0] == self.state[i][1] == self.state[i][2] == a:
                return 1
            elif self.state[0][i] == self.state[1][i] == self.state[2][i] == a:
                return 1
        if self.state[0][0] == self.state[1][1] == self.state[2][2] == a:
            return 1
        if self.state[0][2] == self.state[1][1] == self.state[2][0] == a:
            return 1
        return 0
    
    def is_draw(self, a):
        if self.is_win(a):
            return 0
        if np.all(self.state):
            return 1
        else:
            return 0
        
    def is_done(self):
        if self.is_win(1) or self.is_win(2) or self.is_draw(a):
            return 1
        else:
            return 0
        
        
    def update(self, target, a):
        state = self.state.copy()
        x, y = target//3, target%3
        state[x][y] = a
        return Game(state)

In [ ]:
# def mcs(game):
#     n = 10
#     values = [0] * len(game.empty)
    
#     for i, a in enumerate(game.empty):
#         for _ in range(n):
#             values[i] += -playout(game.update())
    

In [6]:
class Random:
    def __init__(self, status):
        self.status = status
        
        
    def action(self, game):
        return np.random.choice(game.empty)

In [15]:
class Node:
    def __init__(self, state):
        self.state = state # 상태
        self.w = 0 # 보상 누계
        self.n = 0 # 시행 횟수
        self.child_nodes = None  # 자녀 노드 군

    # 국면 가치 계산
    def evaluate(self):
        # 게임 종료 시
        if self.state.is_done():
            # 승패 결과로 가치 취득
            value = -1 if self.state.is_lose() else 0 # 패배 시 -1, 무승부 시 0

            # 보상 누계와 시행 횟수 갱신
            self.w += value
            self.n += 1
            return value

        # 자녀 노드가 존재하지 않는 경우
        if not self.child_nodes:
            # 플레이아웃으로 가치 얻기
            value = playout(self.state)

            # 보상 누계와 시행 횟수 갱신
            self.w += value
            self.n += 1

            # 자녀 노드 전개
            if self.n == 10: 
                self.expand()
            return value

        # 자녀 노드가 존재하는 경우
        else:
            # UCB1이 가장 큰 자녀 노드를 평가해 가치 얻기
            value = -self.next_child_node().evaluate() 

            # 보상 누계와 시행 횟수 갱신
            self.w += value
            self.n += 1
            return value

    # 자녀 노드 전개
    def expand(self):
        self.child_nodes = []
        for action in self.state.empty:
            self.child_nodes.append(Node(self.state.next(action)))

    # UCB1이 가장 큰 자녀 노드 얻기
    def next_child_node(self):
         # 시행 횟수가 0인 자녀 노드 반환
        for child_node in self.child_nodes:
            if child_node.n == 0:
                return child_node

        # UCB1 계산
        t = 0
        for c in self.child_nodes:
            t += c.n
        ucb1_values = []
        for child_node in self.child_nodes:
            ucb1_values.append(-child_node.w/child_node.n+(2*math.log(t)/child_node.n)**0.5)

        # UCB1이 가장 큰 자녀 노드 반환
        return self.child_nodes[argmax(ucb1_values)]    
        
class MCTS:
    def __init__(self, status, game):
        self.game = game
        self.status = status
        self.tree = Node(game)
        
    def playout(self, game):
        if game.is_lose(self.status):
            return -1
        
        if game.is_draw(self.status):
            return 0
        
        
        if game.is_win(self.status):
            return 1
        
        return self.playout(game.update(np.random.choice(game.empty)))
    
    def action(self, game):
        a = self.tree.next_child_node()
        print(a)
        
    
    def make_opp(self):
        if self.count % 2 == 0:
            return self.status
        else:
            return 2 + min(0, 1-self.status)
                

In [16]:
g = Game(init_state)
m = MCTS(2, g)


In [17]:
m.action(g)

TypeError: 'NoneType' object is not iterable

In [19]:
def play(game, m1, m2):
    global score
    while 1:
        a1 = m1.action(game)
        game = game.update(a1, m1.status)
        if game.is_win(m1.status):
            score[m1.status-1] += 1
            return 
        elif game.is_draw(m1.status):
            score[2] += 1
            return 

        a2 = m2.action(game)
        game = game.update(a2, m2.status)
        if game.is_win(m2.status):
            score[m2.status-1] += 1
            return 
        elif game.is_draw(m2.status):
            score[2] += 1
            return 


In [22]:
game = Game(init_state)
m1 = Random(1)
m2 = MCS(2, 10)

In [23]:
score = [0, 0, 0]
for _ in range(1000):
    play(game, m1, m2)
    
score1 = score.copy()
    
for _ in range(1000):
    play(game, m2, m1)

In [24]:
score

[133, 1771, 96]

In [25]:
score1

[113, 814, 73]

In [ ]:
sns.barplot(x = [1, 2], y = score[:2])

In [ ]:
game.state

In [ ]:
# v1: 710 vs 1208
# v2: 113 vs 1771 // 113 vs 814
## 이정도면 완성형
